### Scattering in 1D plane-parallel atmosphere

In [ ]:
using CairoMakie
using FastGaussQuadrature 
using LinearAlgebra
using BenchmarkTools

In [ ]:
"""
Computes the T matrix using Feautrier's method for a 1D plane-parallel atmosphere.
"""
function Tmatrix(tau, μ)
    ndep = length(tau)
    τ = tau / μ
    A = zeros(Float64, ndep)
    B = zeros(Float64, ndep)
    C = zeros(Float64, ndep)

    # Boundary conditions top
    Δτ = τ[2] - τ[1]
    B[1] = 2 / Δτ^2 + 2 / Δτ + 1
    C[1] = 2 / Δτ^2
    # Boundary conditions bottom
    Δτ = τ[end] - τ[end-1]
    A[end] = 2 / (Δτ * (Δτ + 2))
    B[end] = (2 + 2*Δτ + Δτ^2) / (Δτ * (Δτ + 2))

    for i in 2:ndep-1
        Δτ = τ[i+1] - τ[i]
        Δτm = τ[i] - τ[i-1]
        A[i] = 2 / (Δτm * (Δτm + Δτ))
        B[i] = 1 + 2 / (Δτ * Δτm)
        C[i] = 2 / (Δτ * (Δτm + Δτ))
    end

    return Tridiagonal(-A[2:end], B, -C[1:end-1])
end


"""
Returns the nodes and weights form a Gaussian
quadrature with k points. Rescaled to an interval
from [0, 1].
"""
function quadrature(k=5)
    nodes, weights = gausslegendre(k)
    return nodes/2 .+ 0.5, weights/2
end

"""
Solve a coherent scattering problem with direct solver.
"""
function solve_cs_direct(τ, ε, B)
    n = length(τ)
    Λ = zeros(eltype(τ), n, n)
    for (μ, w) in zip(quadrature()...)
        T = Tmatrix(τ, μ)
        Λ .+= w * inv(T)
    end
    M = inv(I - (1 - ε) .* Λ)
    S = ε * M * B
    J = Λ * S
    return S, J
end

In [ ]:
function lambda_matrix(τ; k=5)
    n = length(τ)
    Λ = zeros(eltype(τ), n, n)
    for (μ, w) in zip(quadrature(k)...)
        T = Tmatrix(τ, μ)
        Λ .+= w * inv(T)
    end
    
    return Λ
end

### Thermalization depth